# Analysis Data Fetching
This file contains all necessary code to reproduce the "stages_for_analysis_df.csv" file, containing all stage metadata from procyclingstats. The file takes a few minutes to run if you wish to reproduce the entire dataframe, and it is not necessary to do as the dataframe is already in the environment.

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns
from scipy import stats
import math
import random
import matplotlib.pyplot as plt
import requests
import urllib3
import time
from procyclingstats import *
from bs4 import BeautifulSoup
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [15]:
#Relevant races, their links and if they are a classic or GC
#Races are fetched back to 2018
races = [
     {'name': 'World Championship', 'url_name': 'world-championship', 'type': 'WC'},
     {'name': 'Amstel Gold Race', 'url_name': 'amstel-gold-race', 'type': 'Classic'},
     {'name': 'Milano san Remo', 'url_name': 'milano-sanremo', 'type': 'Classic'},
     {'name': 'Liege Bastone Liege', 'url_name': 'liege-bastogne-liege', 'type': 'Classic'},
     {'name': 'Il Bombardia', 'url_name': 'il-lombardia', 'type': 'Classic'},
     {'name': 'La Fleche Wallone', 'url_name': 'la-fleche-wallone', 'type': 'Classic'},
     {'name': 'Paris Roubaix', 'url_name': 'paris-roubaix', 'type': 'Classic'},
     {'name': 'Ronde van Vlanderen', 'url_name': 'ronde-van-vlaanderen', 'type': 'Classic'},
     {'name': 'Gent Wevelgen', 'url_name': 'gent-wevelgem', 'type': 'Classic'},
     {'name': 'San Sebastian', 'url_name': 'san-sebastian', 'type': 'Classic'},
     {'name': 'Tirreno Adriatico', 'url_name': 'tirreno-adriatico', 'type': 'GC'},
     {'name': 'Tour de France', 'url_name': 'tour-de-france', 'type': 'GC'},
     {'name': 'Giro de Italia', 'url_name': 'giro-d-italia', 'type': 'GC'},
     {'name': 'Vuelta a Espana', 'url_name': 'vuelta-a-espana', 'type': 'GC'},
     {'name': 'Paris-Nice', 'url_name': 'paris-nice', 'type': 'GC'},
     {'name': 'Volta a Catalunya', 'url_name': 'volta-a-catalunya', 'type': 'GC'},
     {'name': 'Criterium du Dauphine', 'url_name': 'dauphine', 'type': 'GC'}
]

In [3]:
#stage_df stores every historical stage, both predicting and resulting, used for analysis
all_stages_df = pd.DataFrame(columns=[
        'race', 'race_name', 'stage_number', 'stage', 'stage_date', 'distance', 'elev_gain', 'climb_urls',
        'startlist_quality', 'profile_score', 'finish_type'])
all_results_df = pd.DataFrame(columns=['race', 'stage', 'stage_rank', 'specialty', 'rider_name',
                                       'rider_age', 'rider_team', 'UCI_points', 'points', 'time'])

In [16]:
#Functions to retrieve all the relevant race course data and store it in a dataframe
def get_stages_for_gc(race):
    #Getting stage course data
    stages = race.stages()
    cols = ['race', 'race_name', 'stage_number', 'stage', 'stage_date', 'distance', 'elev_gain',
            'climb_urls', 'startlist_quality', 'profile_score', 'finish_type']
    stage_df = pd.DataFrame(columns=cols)
    
    result_cols = ['race', 'stage', 'stage_rank', 'specialty', 'rider_name', 'rider_age',
                   'rider_team', 'UCI_points', 'points', 'time']
    result_df = pd.DataFrame(columns=result_cols)
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    for i in range(len(stages)):
        stage_num = i+1
        stage = Stage(stages[i]['stage_url'])
        date = stage.date()
        distance = stage.distance()
        elev_gain = stage.vertical_meters()
        climb_urls = [x['climb_url'] for x in stage.climbs()]
        startlist_quality = stage.race_startlist_quality_score()
        profile_score = stage.profile_score()
        finish_type = stage.won_how()
        results = stage.results()
        
        new_stage = pd.DataFrame([[
            race.relative_url(), race.name(), stage_num, stages[i]['stage_url'], date, distance, elev_gain,
            climb_urls, startlist_quality, profile_score, finish_type]],
        columns=cols)
        
        stage_df = pd.concat([stage_df, new_stage], ignore_index=True)
        
        #Getting results for stage
        result_url = 'https://www.procyclingstats.com/' + race.relative_url() + '/stage-' + str(stage_num)
        #print(result_url)
        response = requests.get(result_url, headers=headers)
        #print(response.status_code)
        result_soup = BeautifulSoup(response.text, 'html.parser')
        if result_soup.select_one('table') is not None:
            stage_results = result_soup.select_one('table').select('tr')
        else:
            stage_results = []
            print(f'No stage results for race{race}')
            
        for j in range(1, len(stage_results)):
            rider_result = [x.text for x in stage_results[j]]
            if len(rider_result) > 10:
                name = rider_result[6]
                name = name.replace(rider_result[8], '').strip()
                time = rider_result[9].replace('"\xa0"', '')

                new_row = pd.DataFrame([[
                    race.relative_url(), stages[i]['stage_url'], rider_result[0], rider_result[5], name,
                    rider_result[7], rider_result[8], rider_result[9], rider_result[10], time
                    ]], columns=result_cols)
    
                result_df = pd.concat([result_df, new_row], ignore_index=True)
            else:
                print(f'Failed fetching results for rider {i} in race {race}')
        
    return stage_df, result_df  

In [17]:
def get_single_day_race_details(race):
    #Getting race course data
    cols = ['race', 'race_name', 'stage_number', 'stage', 'stage_date', 'distance', 'elev_gain',
            'climb_urls', 'startlist_quality', 'profile_score', 'finish_type']
    stage_df = pd.DataFrame(columns=cols)
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    race_url = 'https://www.procyclingstats.com/' + race.relative_url() + '/result/route/profiles'
    response = requests.get(race_url, headers=headers)
    star_soup = BeautifulSoup(response.text, 'html.parser')
    
    info_wrapper = star_soup.select('.infolist')[0]
    race_info = info_wrapper.select('li')
    def get_text(word):
        string = list(filter(lambda x : word in str(x), race_info))[0].select('div')[1].text
        return string

    distance =  float(get_text('Distance:').split(' ')[0])
    elev_gain = int(get_text('Vert. meters:'))
    startlist_quality = int(get_text('Startlist quality'))
    profile_score = int(get_text('ProfileScore:'))
    finish_type = get_text('Won how:')
    
    new_stage = pd.DataFrame([[
        race.relative_url(), race.name(), None, None, race.startdate(), distance, elev_gain,
        [], startlist_quality, profile_score, finish_type
    ]], columns=cols)
    
    stage_df = pd.concat([stage_df, new_stage], ignore_index=True)
    
    #Getting results for race
    result_url = 'https://www.procyclingstats.com/' + race.relative_url() + '/result'
    response = requests.get(result_url, headers=headers)
    result_soup = BeautifulSoup(response.text, 'html.parser')
    
    cols = ['race', 'stage', 'stage_rank', 'specialty', 'rider_name', 'rider_age',
            'rider_team', 'UCI_points', 'points', 'time']
    result_df = pd.DataFrame(columns=cols)
    if result_soup.select_one('table') is not None:
        content = result_soup.select_one('table').select('tr')
    else:
        content = []
        
    for i in range(1, len(content)):
        rider_result = [x.text for x in content[i]]
        if len(rider_result) > 9:
            name = rider_result[4]
            name = name.replace(rider_result[6], '').strip()
            time = rider_result[9].replace(',', '')

            new_row = pd.DataFrame([[
                race.relative_url(), None, rider_result[0], rider_result[3], name, rider_result[5],
                rider_result[6], rider_result[7], rider_result[8], time
                ]], columns=cols)
    
            result_df = pd.concat([result_df, new_row], ignore_index=True)
    
    return stage_df, result_df

In [18]:
def get_race_details(stage_df, results_df, race_url, race_type, race_years):
    races = list()
    for race_year in race_years:
        try:
            race = Race('race/' + race_url + '/' + str(race_year))
            races.append({'race': race, 'year': race_year})
        except:
            print(f"Race {race.name()} in year {race_year} returned Error")
        
    for race in races:
        time.sleep(2)
        print(f"Getting race {race['race'].name()} for year {race['year']}..")
        if race_type == 'GC':
            try:
                stages, results = get_stages_for_gc(race['race'])
                stage_df = pd.concat([stage_df, stages], ignore_index=True)
                results_df = pd.concat([results_df, results], ignore_index=True)
            except:
                print(f"Race {race['race'].name()} in year {race['year']} returned Error")
        else:
            try:
                stage, results = get_single_day_race_details(race['race'])
                stage_df = pd.concat([stage_df, stage], ignore_index=True)
                results_df = pd.concat([results_df, results], ignore_index=True)
            except:
                print(f"Race {race['race'].name()} in year {race['year']} returned Error")
    return stage_df, results_df

In [19]:
for race in races:
    all_stages_df, all_results_df = get_race_details(all_stages_df, all_results_df,
                                                     race['url_name'], race['type'],
                                                     [2018, 2019, 2020, 2021, 2022, 2023])

Getting race World Championships ME - Road Race for year 2018..
Getting race World Championships ME - Road Race for year 2019..
Getting race World Championships ME - Road Race for year 2020..
Getting race World Championships ME - Road Race for year 2021..
Getting race World Championships ME - Road Race for year 2022..
Getting race World Championships ME - Road Race for year 2023..
Getting race Amstel Gold Race for year 2018..
Getting race Amstel Gold Race for year 2019..
Getting race Amstel Gold Race for year 2020..
Race Amstel Gold Race in year 2020 returned Error
Getting race Amstel Gold Race for year 2021..
Getting race Amstel Gold Race for year 2022..
Getting race Amstel Gold Race for year 2023..
Getting race Milano-Sanremo for year 2018..
Getting race Milano-Sanremo for year 2019..
Getting race Milano-Sanremo for year 2020..
Getting race Milano-Sanremo for year 2021..
Getting race Milano-Sanremo for year 2022..
Getting race Milano-Sanremo for year 2023..
Getting race Liège - Basto

In [12]:
all_stages_df.head()

,race,race_name,stage_number,stage,stage_date,distance,elev_gain,climb_urls,startlist_quality,profile_score,finish_type
0,race/world-championship/2018,World Championships ME - Road Race,None,None,2018-09-30,258.0,5020,[],1374,265,Sprint of small group
1,race/world-championship/2019,World Championships ME - Road Race,None,None,2019-09-29,260.7,3220,[],1443,82,Sprint of small group
2,race/world-championship/2020,World Championships ME - Road Race,None,None,2020-09-27,258.2,4662,[],1008,208,12 km solo
3,race/world-championship/2021,World Championships ME - Road Race,None,None,2021-09-26,268.3,2250,[],1357,54,17.4 km solo
4,race/world-championship/2022,World Championships ME - Road Race,None,None,2022-09-25,266.9,4167,[],1052,160,25.8 km solo


In [20]:
all_results_df

,race,stage,stage_rank,specialty,rider_name,rider_age,rider_team,UCI_points,points,time
0,race/tour-de-france/2018,race/tour-de-france/2018/stage-1,1,Sprint,GAVIRIA Fernando,23,Quick-Step Floors,120,100,120
1,race/tour-de-france/2018,race/tour-de-france/2018/stage-1,2,Sprint,SAGAN Peter,28,BORA - hansgrohe,50,70,50
2,race/tour-de-france/2018,race/tour-de-france/2018/stage-1,3,Sprint,KITTEL Marcel,30,Team Katusha Alpecin,25,50,25
3,race/tour-de-france/2018,race/tour-de-france/2018/stage-1,4,Classic,KRISTOFF Alexander,31,UAE Team Emirates,15,40,15
4,race/tour-de-france/2018,race/tour-de-france/2018/stage-1,5,Classic,LAPORTE Christophe,25,"Cofidis, Solutions Crédits",5,32,5
...,...,...,...,...,...,...,...,...,...,...
79829,race/san-sebastian/2023,None,DNF,TT,AZPARREN Xabier Mikel,24,Euskaltel - Euskadi,,,-
79830,race/san-sebastian/2023,None,DNF,Climber,ISASA Xabier,21,Euskaltel - Euskadi,,,-
79831,race/san-sebastian/2023,None,DNF,TT,ELOSEGUI Iñigo,25,Equipo Kern Pharma,,,-
79832,race/san-sebastian/2023,None,DNS,TT,CABOT Jérémy,32,TotalEnergies,,,-


### Do not run the below!!!
Unless you need to.

In [21]:
all_stages_df.to_csv('stages_for_analysis.csv', index=False)
all_results_df.to_csv('results_for_analysis.csv', index=False)